In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import cptac.colon as c

Welcome to the cptac data service package. Available datasets may be
viewed using cptac.list_data(). In order to access a specific data
set, import a cptac subfolder using either 'import cptac.dataset' or
'from cptac import dataset'.
******
Version: 0.4.0
******
You have loaded the cptac colon dataset. To view available dataframes,
use cptac.colon.list_data(). To view available functions for accessing
and manipulating the dataframes, use cptac.colon.list_api().
colon data version: Most recent release

Loading cptac colon data:
Loading clinical data...
Loading miRNA data...
Loading mutation data...
Loading mutation_binary data...
Loading phosphoproteomics_normal data...
Loading phosphoproteomics_tumor data...
Loading proteomics_normal data...
Loading proteomics_tumor data...
Loading transcriptomics data...


In [2]:
somatic_mutation = c.get_mutations()
sample_status_map = c.get_sample_status_map()
transcriptomics = c.get_transcriptomics()
proteomics = c.get_proteomics()
gene = 'PTEN'
omics_mutations = c.append_mutations_to_omics(mutation_genes=gene, omics_df=proteomics, omics_genes=gene)

In [34]:
sample_status_map = c.get_sample_status_map()
merged_mutations = somatic_mutation.join(sample_status_map, how="left")

In [8]:
# get total tumors/patients
sample_status_series = sample_status_map.value_counts()
total_tumor_patients = sample_status_series[0]
print('total_tumor_patients', total_tumor_patients)

total_tumor_patients 110


In [61]:
freq_mut = ['APC','TTN','TP53']
truncated = []
for gene in freq_mut:
    gene_mutated = merged_mutations.loc[merged_mutations['Gene'] == gene]
    trunc = gene_mutated.loc[(gene_mutated['Mutation'] != 'nonframeshift deletion') & 
        (gene_mutated['Mutation'] != 'nonframeshift insertion') & 
        (gene_mutated['Mutation'] != 'nonframeshift substitution') & 
        (gene_mutated['Mutation'] != 'nonsynonymous SNV')] 
    #print(trunc.head())
    print(trunc['Mutation'].value_counts())
    num_gene_mutated = len(trunc)
    percentage = (num_gene_mutated / total_tumor_patients)
    print('\n', gene,num_gene_mutated, '/',total_tumor_patients, '\n')
    truncated.append(percentage)

stopgain                   182
frameshift deletion         77
frameshift insertion        65
frameshift substitution     21
Name: Mutation, dtype: int64

 APC 345 / 110 

stopgain                   69
frameshift deletion        51
frameshift insertion       29
frameshift substitution     6
Name: Mutation, dtype: int64

 TTN 155 / 110 

stopgain                147
frameshift deletion      74
frameshift insertion     15
Name: Mutation, dtype: int64

 TP53 236 / 110 



In [62]:
truncated

[3.1363636363636362, 1.4090909090909092, 2.1454545454545455]

In [60]:
freq_mut = ['TP53']
missence = []
for gene in freq_mut:
    count = 0
    gene_mutated = merged_mutations.loc[merged_mutations['Gene'] == gene]
    #print(gene_mutated.head())
    miss_mutated = gene_mutated.loc[gene_mutated['Mutation'] == 'nonframeshift deletion']
    print('\n','deletion')
    print(miss_mutated['Mutation'].value_counts())
    #print(miss_mutated.head())
    count += len(miss_mutated)
    miss_mutated = gene_mutated.loc[(gene_mutated['Mutation'] == 'nonframeshift insertion')]
    count += len(miss_mutated)
    print('\n','insertion')
    print(miss_mutated['Mutation'].value_counts())
    #print(miss_mutated.head())
    miss_mutated = gene_mutated.loc[(gene_mutated['Mutation'] == 'nonframeshift substitution')]
    count += len(miss_mutated)
    print('\n','substitution')
    print(miss_mutated['Mutation'].value_counts())
    #print(miss_mutated.head())
    miss_mutated = gene_mutated.loc[(gene_mutated['Mutation'] == 'nonsynonymous SNV')]
    count += len(miss_mutated)
    print('\n','SNV')
    print(miss_mutated['Mutation'].value_counts())
    #print(miss_mutated.head())
    
    num_gene_mutated = count
    percentage = (num_gene_mutated / total_tumor_patients)
    print('\n', gene, num_gene_mutated, '/',total_tumor_patients)
    missence.append(percentage)
    print('percentage:',percentage)


 deletion
nonframeshift deletion    7
Name: Mutation, dtype: int64

 insertion
Series([], Name: Mutation, dtype: int64)

 substitution
Series([], Name: Mutation, dtype: int64)

 SNV
nonsynonymous SNV    526
Name: Mutation, dtype: int64

 TP53 533 / 110
percentage: 4.845454545454546


In [87]:
#merge = transcriptomics.join(somatic_mutation, how = "left")
merge = somatic_mutation.join(sample_status_map, how="left") 
merge = merge.loc[merge['Gene'] == 'APC']
merge = merge.loc[merge['Sample_Status'] == 'Tumor']
len(merge.index.unique())
print(merge.head())
sample_status_series = sample_status_map.value_counts()
print(sample_status_series)
total_tumor_patients = sample_status_series[0]
total_tumor_patients

          Gene           Mutation      Location Sample_Status
Sample_ID                                                    
S001       APC  Nonsense_Mutation       p.R232*         Tumor
S001       APC    Frame_Shift_Ins  p.T1556Nfs*3         Tumor
S001       APC  Missense_Mutation      p.D2796Y         Tumor
S010       APC    Frame_Shift_Del  p.K1878Rfs*4         Tumor
S012       APC  Missense_Mutation      p.S2029Y         Tumor
Tumor     95
Normal    49
Name: Sample_Status, dtype: int64


95

In [88]:
PTEN = merge.loc[merge['Gene'] == 'PTEN']
#len(PTEN.index.unique())
print('total_tumor_patients:', total_tumor_patients)

total_tumor_patients: 95


In [89]:
p = merge.loc[merge['Gene'] == 'APC']
print('total mut:', len(p))
trunc = p.loc[p['Mutation'] != 'In_Frame_Del']
trunc = trunc.loc[trunc['Mutation'] != 'In_Frame_Ins']
trunc = trunc.loc[trunc['Mutation'] != 'Missense_Mutation']
print(trunc['Mutation'].unique())
print('num trunc:', len(trunc))
num_gene_mutated = len(trunc)
percentage = (num_gene_mutated / total_tumor_patients)
print("percent trunc", percentage)

total mut: 27
['Nonsense_Mutation' 'Frame_Shift_Ins' 'Frame_Shift_Del']
num trunc: 13
percent trunc 0.1368421052631579


In [73]:
count = 0 
m = p.loc[p['Mutation'] == 'In_Frame_Del']
count = len(m)
m = p.loc[p['Mutation'] == 'In_Frame_Ins']
count += len(m)
m = p.loc[p['Mutation'] == 'Missense_Mutation']
count += len(m)
print('num miss:', count)
num_gene_mutated = count
percentage = (num_gene_mutated / total_tumor_patients)
print("percent miss", percentage)

num miss: 0
percent miss 0.0


In [ ]:
trunc = p.any()

In [54]:
n = p.loc[p['Mutation'] != 'In_Frame_Del']
n = p.loc[p['Mutation'] != 'In_Frame_Ins']
n = p.loc[p['Mutation'] != 'Missense_Mutation']

In [56]:
n["Mutation"].unique()

array(['Nonsense_Mutation', 'Frame_Shift_Ins', 'Frame_Shift_Del',
       'Splice_Site'], dtype=object)

In [5]:
lst = ['pten', 'pik3ca']
lst2 = [.79,.45]
lst3 = [.66,.22]


df = pd.DataFrame(list(zip(lst,lst2,lst3)), columns = ['1','2','3'])


In [6]:
df

,1,2,3
0,pten,0.79,0.66
1,pik3ca,0.45,0.22
